In [1]:
from data import Data
import numpy as np
DATA_DIR = 'data/'

train = Data(fpath = DATA_DIR + 'train.csv')
test = Data(fpath = DATA_DIR + 'test.csv')

train
train.attributes['cap-color'].possible_vals
S = np.array(train.get_column(['label', 'cap-color']))
print(S)
data = np.loadtxt(DATA_DIR + 'train.csv', delimiter=',', dtype = str)
print(data)
cool_data = Data(data=data)

new = Data(data=S)
# yeet = [1,2,3,4,5,6,7,8,9,10]
# yeet = np.array(yeet)
# print(yeet)

[['p' 'n']
 ['e' 'w']
 ['p' 'w']
 ...
 ['e' 'n']
 ['e' 'n']
 ['e' 'n']]
[['label' 'cap-shape' 'cap-surface' ... 'spore-print-color' 'population'
  'habitat']
 ['p' 'x' 's' ... 'k' 's' 'u']
 ['e' 'b' 's' ... 'n' 'n' 'm']
 ...
 ['e' 'k' 's' ... 'b' 'c' 'l']
 ['e' 'f' 's' ... 'b' 'c' 'l']
 ['e' 'x' 's' ... 'o' 'c' 'l']]


In [2]:

train_label_col = train.get_column('label')
test_label_col = test.get_column('label')

def label_count(col):
    e = 0
    p = 0
    for val in col:
        if val == 'e':
            e += 1
        else:
            p += 1
    return e,p


print(f'train rows: {train_label_col.__len__()}')
e,p = label_count(train_label_col)
print('e:', e)
print('p:', p)
majority_label_train = e

print(f'test rows: {test.__len__()}')
e,p = label_count(test_label_col)
print('e:', e)
print('p:', p)
majority_label_test = e



train rows: 6530
e: 3382
p: 3148
test rows: 1594
e: 826
p: 768


1.) So e is the most common label in the training data. If a classifier always picks this label (e), the accuracy would be 3382/6530 = 51.8%. For the test data, a classifier that always picks e would be 826/1594 = 51.8%. 

In [3]:



class Node: 
    def __init__(self, attribute):
        self.attribute = attribute
        self.branches = []
        self.label = None
        
    def add_branch(self, node):
        self.branches.append(node)
        
# S = set of examples
# attributes = set of of measured attributes        
def ID3(S, attributes):
    root = Node(None)
    # find attribute in attributes that best classifies S
    A = bestAttribute(S, attributes)
    possible_vals = A.get_attribute_possible_vals()
    for val in possible_vals:
        newNode = Node(None)
        root.add_branch(newNode)
        Sv = S.get_row_subset(A, val)
        if Sv.__len__() == 0:
            newNode.label = majority_label
        else:
            attributes.remove(A)
            newNode = ID3(Sv, attributes)
    return root

def col_label_count(colName):
    e = 0
    p = 0
    possible_vals = train.get_attribute_possible_vals(colName)
    vals_dict = {}
    # with_label = train.get_column(['label', colName])
    # print(with_label)
    # print(with_label.attributes)
    # with_label = Data(data=train.get_column(['label', colName]))
    # print(with_label.get_row_subset(colName, val))


    
    for val in possible_vals :
        vals_dict[val] = [0,0]
    
    for val in possible_vals :   
        print(train.get_row_subset(colName, val))
        print(Data(data=train.get_row_subset(colName, val).raw_data))
        yeet = Data(data=train.get_row_subset(colName, val).raw_data)
        print(yeet.raw_data)
        print(yeet.attributes)
        yeet.raw_data.get_row_subset('label', 'e')
        # Data(data=train.get_row_subset(colName, val).raw_data).get_row_subset('label', 'e').get_row_count()
        Data(data=train.get_row_subset(colName, val).raw_data).get_row_subset('label', 'e').get_row_count()

        vals_dict[val][0] = Data(data=train.get_row_subset(colName, val).raw_data).get_row_subset('label', 'e').get_row_count()  
        vals_dict[val][1] = train.get_row_count() - vals_dict[val][0]
    print('survived col_label_count')
    return vals_dict
    

def entropy(num_e, num_p) :
    total = num_e + num_p
    return -((num_e/total) * np.log2(num_e/total)) - ((num_p/total) * np.log2(num_p/total))

# Information gain using entropy
def col_entropy(col):
    total_e = 0
    label_counts = col_label_count(col)
    num_rows = col.__len__()
    for val in label_counts : 
        num_vals = label_counts[val][0] + label_counts[val][1]
        total_e += entropy(label_counts[val][0], label_counts[val][1]) * (num_vals/num_rows)

def info_gain(S, attribute):
    e,p = col_label_count(attribute)
    if e == 0 or p == 0:
        return 0
    else:
        return entropy(S) - col_entropy(attribute)
    
def bestAttribute(S, attributes):
    best = None
    best_gain = 0
    for a in attributes:
        gain = info_gain(S, a)
        if gain > best_gain:
            best_gain = gain
            best = a
    return best
    
    
# find tree with ID3 algorithm

# - start with S (all the rows/data basically) and attricutes (all the columns/attributes)

majority_label = majority_label_train

decision_tree = ID3(train, train.attributes)

[['e' 'b' 's' ... 'k' 'n' 'm']
 ['e' 'b' 'y' ... 'n' 's' 'm']
 ['e' 'b' 's' ... 'k' 's' 'm']
 ...
 ['e' 'b' 'f' ... 'w' 's' 'g']
 ['e' 'b' 's' ... 'w' 'n' 'g']
 ['e' 'b' 's' ... 'w' 'n' 'g']]
{'e': <data.Attribute object at 0x0000017C609B1690>, 'b': <data.Attribute object at 0x0000017C609B1150>, 's': <data.Attribute object at 0x0000017C52289F10>, 'w': <data.Attribute object at 0x0000017C60C9E390>, 't': <data.Attribute object at 0x0000017C609B1390>, 'f': <data.Attribute object at 0x0000017C609B1310>, 'c': <data.Attribute object at 0x0000017C60C66DD0>, 'n': <data.Attribute object at 0x0000017C522896D0>, 'p': <data.Attribute object at 0x0000017C60C9F0D0>, 'o': <data.Attribute object at 0x0000017C609B13D0>, 'm': <data.Attribute object at 0x0000017C609B1050>}


AttributeError: 'numpy.ndarray' object has no attribute 'get_row_subset'